In [1]:
import cv2
import numpy as np
import time 
from PIL import Image
from picamera.array import PiRGBArray
from picamera import PiCamera

In [2]:
thres = 0.6 # Threshold to detect object
nms_threshold = 0.2
 
classNames= []
classFile = 'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n') 
print(classNames)


['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'street sign', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'hat', 'backpack', 'umbrella', 'shoe', 'eye glasses', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'plate', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'mirror', 'dining table', 'window', 'desk', 'toilet', 'door', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'blender', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush', 'hair brush']


In [3]:

configPath = 'ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt'
weightsPath = 'frozen_inference_graph.pb'
 
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(128,128)
#net.setInputSize(320,320)
net.setInputScale(1.0/ 127.5)
net.setInputMean((127.5, 127.5, 127.5))
net.setInputSwapRB(True)



<dnn_Model 0xb47e8be0>

In [4]:
from picamera.array import PiRGBArray
from picamera import PiCamera

# initialize the camera and grab a reference to the raw camera capture
prev_frame_time = 0 
# used to record the time at which we processed current frame
new_frame_time = 0
camera = PiCamera()
camera.resolution = (640, 480)
#camera.resolution = (320, 240)
camera.framerate = 30
rawCapture = PiRGBArray(camera, size=(640, 480))
#rawCapture = PiRGBArray(camera, size=(320, 240))

# allow the camera to warmup
time.sleep(0.1)
# capture frames from the camera
for fram in camera.capture_continuous(rawCapture, format="bgr", use_video_port=True):
	# grab the raw NumPy array representing the image, then initialize the timestamp
	# and occupied/unoccupied text
	frame = fram.array
	orig = frame.copy()
	classIds, confs, bbox = net.detect(frame,confThreshold=thres)   
	indices= cv2.dnn.NMSBoxes(bbox,confs, thres,  nms_threshold)
	
	for classId, confidence, box in zip (classIds, confs, bbox):
		cv2.rectangle(orig, box, color=(0, 255, 0), thickness=1)
		cv2.putText(orig, classNames[classId-1], (box[0]+10, box[1]+30), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0))
		cv2.putText(orig, str("{0:.1f}".format(confidence*100)), (box[0]+15, box[1]+50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0))

	
	#font = cv2.FONT_HERSHEY_SIMPLEX
	new_frame_time = time.time() 
	fps = 1/(new_frame_time-prev_frame_time)
	prev_frame_time = new_frame_time 
	fps = float(fps) 
	fps = str("{0:.1f}".format(fps))
	cv2.putText(orig, fps , (10, 10), cv2.FONT_HERSHEY_COMPLEX, 1, (100, 255, 0), 1, cv2.LINE_AA)
	print(fps)
	
	cv2.imshow('video2', orig)
	rawCapture.truncate(0)
	if cv2.waitKey(1) & 0xFF == ord('q'): # Se esperan 30ms para el cierre de la ventana o hasta que el usuario precione la tecla q
		break


	

cv2.destroyAllWindows()

0.0
1.7
3.1
3.9
3.5
4.1
3.7
2.7
4.8
2.8
2.9
4.2
3.2
2.8
3.8
3.9
3.0
3.5
3.4
3.8
4.2
3.2
4.5
3.2
4.7
2.8
3.8
3.1
3.6
3.7
5.4
4.1
3.6
3.6
3.2
4.2
4.8
4.1
4.5
3.8
4.4
4.7
5.0
4.0
4.3
4.5
4.3
3.4
4.2
3.6
3.6
4.9
3.2
3.5
3.4
4.2
4.0
4.0
4.6
4.1
4.1
3.6
3.8
5.0
4.2
4.7
4.2
4.9
4.1
5.1
4.6
4.6
4.9
4.6
5.1
4.7
3.7
3.4
5.3
4.3
4.6
5.6
3.7
4.3
4.0
4.0
3.7
4.2
3.2
3.8
5.4
4.8
4.7
4.5
4.3
5.1
4.4
4.2
5.0
3.2
4.4
5.3
4.5
4.2
4.5
3.8
3.8
5.2
4.3
5.5
3.2
3.4
5.3
3.8
4.5
3.6
5.1
5.2
5.5
4.7
3.7
5.6
4.7
5.0
4.7
2.9
4.3
6.8
4.4
5.1
4.7
4.3
5.1
4.9
5.4
4.5
4.1
5.8
4.3
5.3
5.0
5.0
5.2
3.9
5.8
4.4
4.6
5.0
4.4
6.4
5.0
5.6
4.6
4.0
7.0
4.4
5.8
5.4
5.1
4.8
4.9
6.1
4.0
5.4
4.5
3.7
3.0
4.5
3.7
4.2
3.9
3.8
3.5
4.8
4.9
4.8
3.8
4.1
4.7
4.4
4.6
4.5
4.0
4.8
3.4
4.9
3.4
2.7
4.0
4.5
4.2
4.8
4.6
4.7
4.5
3.9
4.7
3.8
2.5
2.8
4.2
3.3
3.9
3.8
3.9
4.0
4.0
4.8
2.5
4.1
3.8
4.3
3.7
5.3
3.9
4.6
5.2
3.3
4.5
4.1
4.1
2.9
4.7
4.0
4.7
4.5
5.3
4.3
4.1
4.4
4.2
4.5
3.7
4.2
3.5
3.9
4.4
4.2
3.4
2.9
2.7
4.4
3.3
2.6
2.8
2.2
2.2
3.6
5.0
4.4
